## First week

>

## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

Ans: Siri, Google translation, automater drive

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: Github simplifies the process of working with other people and makes it easy to collaborate on projects. Jupyter notebook file are structured as executable cells.We can use smart shortcuts to execute the current cell. Pycharm can also work with Jupyter notebook files.

#### 2. What's the Probability Model?

Ans:a mathematical representation to computer the probability

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans: Predict whether it will rain

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans: We need to realize all the patterns, which will be difficult when we try to program based on parsing and pattern match. Probability will simplify the task. 

#### 5. What's the Language Model;

Ans:is a probability distribution over sequences of words. Given such a sequence, say of length m, it assigns a probability P(w1,w2...wn) to the whole sequence.

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans: MT, POS, parsing

#### 7. What's the 1-gram language model;

Ans: An n-gram is a contiguous sequence of n items from a given sample of text. So 1 means size is one. 

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:It is easy to compute the probability of one word. However, in fact words are not independent and they have connections with context information.

#### 9. What't the 2-gram models;

Ans: An n-gram is a contiguous sequence of n items from a given sample of text. So we use two contiguous words together. 

## 编程实践部分

#### 1. 根据语法规则写出句子生成器

In [59]:
import random
choice = random.choice

def create_grammar(grammar_str, split = '=', line_split = '\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [ s.split() for s in stmt.split('|') ]
    return grammar

def generate(gram, target):
    if target not in gram: return target   # it means target is terminal expression

    expand = [generate(gram, t) for t in choice(gram[target]) ]
    return ''.join(e if e != '/n' else '\n' for e in expand if e != 'null') 

# the first grammar
teacher = """
order = 名字 状态 动作 题号 结尾 
名字 = 张三 | 李四 | 王五
状态 = 站着 | 坐着 | 上来
动作 = 解答
题号 = 第 数字 题
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
结尾 = 。
"""

# the second grammar
student = """
answer = 回答 断句 尊称 断句 题号 答案 结尾
回答 = 到 | 在
断句 = ，
尊称 = 姓 老师
姓 = 于 | 张 | 黄
题号 = 第 数字 题
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
答案 = 我 结果
结果 = 会 | 不会
结尾 = 。
"""

In [60]:
generate(gram=create_grammar(teacher, split='='), target='order')

'张三站着解答第9题。'

In [61]:
generate(gram=create_grammar(student, split='='), target='answer')

'到，于老师，第8题我会。'

In [148]:
def generate_n(n):
    sents =[]
    for i in range(n):
        #print(generate(gram=create_grammar(teacher, split='='), target='order'))
        sen = generate(gram=create_grammar(student, split='='), target='answer')
        sents.append(sen)
    return sents


In [149]:
generate_n(10)

['在，张老师，第55613题我不会。',
 '到，张老师，第961题我会。',
 '在，黄老师，第187题我不会。',
 '到，于老师，第6题我不会。',
 '在，于老师，第84题我会。',
 '到，于老师，第67815题我不会。',
 '在，于老师，第6题我会。',
 '到，黄老师，第79题我会。',
 '在，于老师，第2题我不会。',
 '到，于老师，第45题我会。']

#### 2. 使用新数据源完成语言模型的训练

按照我们上文中定义的`prob_2`函数，我们更换一个文本数据源，获得新的Language Model:

1. 下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）
    + 可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv
2. 修改代码，获得新的**2-gram**语言模型
    + 进行文本清洗，获得所有的纯文本
    + 将这些文本进行切词
    + 送入之前定义的语言模型中，判断文本的合理程度

In [80]:
import pandas as pd
import re 
import jieba

In [81]:
filename = '/Users/sun/Desktop/hk/datasource-master/movie_comments.csv'
content = pd.read_csv(filename, encoding='utf-8', low_memory=False)

In [82]:
content.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [83]:
articles = content['comment'].tolist()

In [84]:
len(articles)

261497

In [85]:
def token(string):
    return re.findall('\w+', string)

In [86]:
articles_clean = [''.join(token(str(a)))for a in articles]

In [87]:
def cut(string): return list(jieba.cut(string))

In [88]:
articles_words = [
    cut(string) for string in articles_clean[:100000]
]

In [89]:
TOKENS = []

In [90]:
from functools import reduce

In [91]:
from operator import add

In [92]:
TOKENS = reduce(add, articles_words)

In [94]:
from collections import Counter

In [95]:
words_count = Counter(TOKENS)

In [96]:
words_count.most_common(10)

[('的', 127398),
 ('了', 44535),
 ('是', 29955),
 ('我', 21014),
 ('都', 15430),
 ('看', 14093),
 ('电影', 13545),
 ('也', 13437),
 ('很', 13356),
 ('和', 12586)]

In [97]:
frequience = [f for w,f in words_count.most_common()]

In [98]:
def prob_1(word):
    return words_count[word]/len(TOKENS)

In [99]:
prob_1('我们')

0.0011781812112071372

In [100]:
TOKENS = [str(t) for t in TOKENS]

In [101]:
TOKEN_2_GRAM = [''.join(TOKENS[i:i+2]) for i in range(len(TOKENS[:-2]))]

In [102]:
words_count_2 = Counter(TOKENS)

In [103]:
def prob_2(word1,word2):
    if word1 + word2 in words_count: return words_count_2[word1+word2]/len(TOKEN_2_GRAM)
    else: 
        return (prob_1(word1) + prob_1(word2))/2

In [104]:
prob_2('今天','天气')

9.565270642965028e-05

In [105]:
def get_probability(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        
        probability = prob_2(word,next_)
        
        sentence_pro *= probability
        
    return sentence_pro
        
    

In [106]:
get_probability('今天天气真好')

1.9962863923611288e-08

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



In [190]:
def generate_best(n): # you code here
    sents = generate_n(n)
    
    result = []
    for i in range(len(sents)):
        val = get_probability(sents[i])
        print('{}: {}'.format(sents[i],val))
        result.append((val,sents[i])) 
    val,sen = sorted(result, key=lambda x:x[0],reverse=True)[0]
    #print(result)
    print("The highest probability sentence is: {}  {} ".format(sen,val))

    
    return  # return the sentences which has the highest probability

In [191]:
generate_best(10)

到，于老师，第365题我不会。: 7.104128924617566e-37
到，于老师，第4题我不会。: 2.134517506119709e-34
到，张老师，第2659题我不会。: 0.0
在，于老师，第364835题我会。: 3.555907236291636e-37
到，张老师，第78题我会。: 0.0
在，张老师，第1题我不会。: 0.0
到，黄老师，第2394题我不会。: 1.188450679904578e-37
到，黄老师，第6题我会。: 1.5863545027183425e-36
在，黄老师，第9题我会。: 1.8885627534437843e-36
在，黄老师，第8题我会。: 3.581898266247493e-36
The highest probability sentence is: 到，于老师，第4题我不会。  2.134517506119709e-34 


#### 这个模型有什么问题？ 准备如何提升？ 

如果有一个2元组在corpus中没出现的话，那么整句话的概率都为0，实际概率并不为0。

处理方法：平滑处理。

##### 以下内容为可选部分，对于绝大多数同学，能完成以上的项目已经很优秀了，下边的内容如果你还有精力可以试试，但不是必须的。

#### 4. (Optional) 完成基于Pattern Match的语句问答
> assignment-01-optional-pattern-match：本周时间不充足，下周在做。


#### 5. (Optional) 完成阿兰图灵机器智能原始论文的阅读
1. 请阅读阿兰图灵关于机器智能的原始论文：https://github.com/Computing-Intelligence/References/blob/master/AI%20%26%20Machine%20Learning/Computer%20Machinery%20and%20Intelligence.pdf 
2. 并按照GitHub仓库中的论文阅读模板，填写完毕后发送给我: mqgao@kaikeba.com 谢谢

> 